# ДЗ 9: Information Extraction из новостных диалогов

**Трек B** — извлечение сущностей (PERSON, ORG, LOC, EVENT, DATE, IMPACT, SOURCE) из диалогов с помощью LLM.

**Этапы:**
1. Локальное развертывание моделей (quantized vs full)
2. Подготовка данных (WildChat-1M)
3. Оптимизация для IE (batch processing)
4. Анализ производительности

## 0. Установка зависимостей

> В Colab: **Runtime → Change runtime type → GPU (T4)**. После pip может понадобиться Restart.

In [ ]:
# Без pipeline → меньше зависимостей, нет ошибки torchvision::nms
!pip install -q -U transformers accelerate bitsandbytes datasets

## 1. Загрузка данных (WildChat-1M)

Датасет: 1M диалогов человек–ChatGPT. Подвыборка 500–1K для CPU, 1K–2K для GPU. Demo: 100–200.

In [ ]:
from datasets import load_dataset

N_SAMPLES = 200  # Demo: 200. Для полного: 1000 или 2000

# split='train[:N]' загружает только N примеров (экономит время и диск)
ds = load_dataset("allenai/WildChat-1M", split=f"train[:{N_SAMPLES}]")

def get_conversation_text(sample):
    conv = sample.get("conversation", [])
    parts = []
    for turn in conv:
        c = turn.get("content", "")
        if c:
            parts.append(c.strip())
    return " \n ".join(parts) if parts else ""

texts = [get_conversation_text(ds[i]) for i in range(len(ds))]
texts = [t for t in texts if len(t) > 50]  # фильтр коротких
print(f"Загружено {len(texts)} диалогов")

## 2. Промпт для IE

Извлечение сущностей в JSON: PERSON, ORG, LOC, EVENT, DATE, IMPACT, SOURCE.

In [ ]:
IE_PROMPT = """Extract entities from the text. Return JSON with keys: PERSON, ORG, LOC, EVENT, DATE, IMPACT, SOURCE. Each key is a list of strings. If nothing found, use empty list []. Output ONLY valid JSON, no other text.

Text:
{text}

JSON:"""

def make_ie_prompt(text, max_chars=1500, model_type="mistral"):
    t = text[:max_chars] if len(text) > max_chars else text
    body = IE_PROMPT.format(text=t)
    if model_type == "mistral":
        return f"<s>[INST] {body} [/INST]"
    if model_type == "tinyllama":
        return f"<|system|>\nYou are a helpful assistant.<|user|>\n{body}<|assistant|>\n"
    return body

## 3. Модели: TinyLlama (full/4-bit) и Mistral (4-bit)

Сравнение: quantized vs full precision по скорости и памяти.

In [ ]:
import torch
# Используем только Auto* — без pipeline, чтобы избежать torchvision::nms в Colab
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

def load_model(name, use_4bit=False):
    tokenizer = AutoTokenizer.from_pretrained(name)
    tokenizer.pad_token = tokenizer.eos_token
    if use_4bit:
        bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
        model = AutoModelForCausalLM.from_pretrained(name, quantization_config=bnb, device_map="auto")
    else:
        model = AutoModelForCausalLM.from_pretrained(name, device_map="auto", torch_dtype=torch.float16)
    return model, tokenizer

# Конфиг: какая модель (для Colab T4 — tiny 4-bit или mistral 4-bit)
USE_MISTRAL = True  # True = Mistral 4-bit (медленнее, качественнее), False = TinyLlama

if USE_MISTRAL:
    model_id = "mistralai/Mistral-7B-Instruct-v0.2"
    model, tokenizer = load_model(model_id, use_4bit=True)
else:
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model, tokenizer = load_model(model_id, use_4bit=False)

# Вместо pipeline используем model.generate() напрямую
def generate(pipe_model, pipe_tokenizer, prompt, max_new_tokens=256):
    inputs = pipe_tokenizer(prompt, return_tensors="pt").to(pipe_model.device)
    out = pipe_model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, pad_token_id=pipe_tokenizer.eos_token_id)
    return pipe_tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

pipe = (model, tokenizer)  # (model, tokenizer) для совместимости с extract_entities

## 4. IE: единичный и batch

In [ ]:
import time
import json
import re

def extract_entities(pipe, text, max_new_tokens=200, model_type="mistral"):
    model, tokenizer = pipe
    prompt = make_ie_prompt(text, model_type=model_type)
    raw = generate(model, tokenizer, prompt, max_new_tokens=max_new_tokens).strip()
    # Извлекаем первый полный JSON-объект (модель может добавить пояснения после)
    start = raw.find("{")
    if start >= 0:
        depth, end = 0, None
        for i, c in enumerate(raw[start:], start):
            if c == "{": depth += 1
            elif c == "}":
                depth -= 1
                if depth == 0:
                    end = i + 1
                    break
        if end is not None:
            try:
                return json.loads(raw[start:end])
            except json.JSONDecodeError:
                pass
    return {"raw": raw}

def run_ie_batch(pipe, texts, batch_size=1, model_type="mistral"):
    results = []
    t0 = time.time()
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        for t in batch:
            r = extract_entities(pipe, t, model_type=model_type)
            results.append(r)
    elapsed = time.time() - t0
    return results, elapsed

In [ ]:
# Запуск на подвыборке (10 для demo)
N_RUN = min(10, len(texts))
sample_texts = texts[:N_RUN]

model_type = "mistral" if USE_MISTRAL else "tinyllama"
results, elapsed = run_ie_batch(pipe, sample_texts, model_type=model_type)
print(f"Обработано {N_RUN} диалогов за {elapsed:.1f} сек")
print(f"Throughput: {N_RUN/elapsed:.2f} диалогов/сек")
print()
print("Пример извлечения:")
print(json.dumps(results[0], ensure_ascii=False, indent=2)[:500])

## 5. Анализ производительности

- Скорость: диалогов/сек
- Ресурсы: VRAM (torch.cuda)

In [ ]:
if torch.cuda.is_available():
    vram_gb = torch.cuda.max_memory_allocated() / 1e9
    print(f"VRAM пик: {vram_gb:.2f} GB")
print(f"Время на {N_RUN} диалогов: {elapsed:.1f} сек")
print(f"Среднее: {elapsed/N_RUN:.2f} сек/диалог")

## 5.1 Анализ результатов IE

Сводная статистика: доля валидного JSON, распределение сущностей по типам, примеры «вход → выход».

In [ ]:
# Статистика: валидный JSON vs сырой вывод (когда парсер не смог извлечь JSON)
valid_count = sum(1 for r in results if "raw" not in r)
has_raw = [i for i, r in enumerate(results) if "raw" in r]
print(f"Валидный JSON: {valid_count}/{len(results)} ({100*valid_count/len(results):.0f}%)")
print(f"Сырой вывод (модель дала не-JSON или битый JSON): {len(has_raw)}")
if has_raw:
    print("Индексы:", has_raw[:5], "..." if len(has_raw) > 5 else "")

In [ ]:
# Распределение сущностей по типам (среди валидных)
entity_keys = ["PERSON", "ORG", "LOC", "EVENT", "DATE", "IMPACT", "SOURCE"]
counts = {k: 0 for k in entity_keys}
total_entities = 0
for r in results:
    if "raw" in r:
        continue
    for k in entity_keys:
        vals = r.get(k, [])
        if isinstance(vals, list):
            n = len(vals)
        else:
            n = 1 if vals else 0
        counts[k] += n
        total_entities += n

print("Сущностей по типам:")
for k, v in sorted(counts.items(), key=lambda x: -x[1]):
    print(f"  {k}: {v}")
print(f"Всего извлечено: {total_entities}")

In [ ]:
# Примеры: входной текст → извлечённые сущности (2–3 примера)
indices = [0]
if len(results) > 1:
    indices.append(1)
if len(results) > 3:
    indices.append(len(results) // 2)
for idx in indices:
    if idx >= len(sample_texts):
        break
    print("=" * 60)
    print(f"Пример {idx+1}")
    print("-" * 40)
    txt = sample_texts[idx]
    print("Вход (сокращённо):", txt[:300] + "..." if len(txt) > 300 else txt)
    print()
    r = results[idx]
    if "raw" in r:
        print("Вывод (raw):", r["raw"][:400] + "..." if len(r["raw"]) > 400 else r["raw"])
    else:
        print("Извлечено:")
        for k in entity_keys:
            vals = r.get(k, [])
            if vals:
                print(f"  {k}: {vals}")
    print()

In [ ]:
# Итоговая таблица производительности
print("Итоги производительности")
print("-" * 40)
print(f"Модель: {model_id}")
print(f"Диалогов обработано: {N_RUN}")
print(f"Время: {elapsed:.1f} сек")
print(f"Скорость: {N_RUN/elapsed:.3f} диалогов/сек (~{elapsed/N_RUN:.1f} сек/диалог)")
if torch.cuda.is_available():
    print(f"VRAM пик: {torch.cuda.max_memory_allocated()/1e9:.2f} GB")

## 5.2 Что смотреть и возможные улучшения

| Что проверять | Зачем |
|---------------|-------|
| Доля валидного JSON | Низкая → добавить в промпт «только JSON, без текста после» |
| PERSON/ORG/LOC чаще пустые | Диалоги могут не содержать явных сущностей; попробовать другой датасет или фильтр |
| Модель пишет пояснения после JSON | Явно указать: «Output ONLY valid JSON, no extra text» |
| Очень медленно | TinyLlama быстрее; batch-запросы (несколько текстов в одном промпте) |

In [ ]:
# Если много raw — попробуй добавить в IE_PROMPT: "Output ONLY valid JSON, no other text."

## 6. Опционально: вторая модель для сравнения

Перед запуском — освободить память (`del model`, `torch.cuda.empty_cache()`).

In [ ]:
# Сравнение TinyLlama vs Mistral (запускать по очереди, не одновременно)
# 1) TinyLlama ~2GB VRAM, быстрее
# 2) Mistral 4-bit ~6GB VRAM, качественнее
# Замерь время и VRAM для каждой.

## 7. Опционально: системный промпт «когнитивный дизайнер»

Для объяснений в стиле когнитивного дизайнера — см. `prompt_cognitive_designer.md` или `../promt.md`. Добавь в начало промпта перед запросом пользователя.